In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_top_100_at_date(date_str):
    """
    Get top 100 cryptocurrencies for a specific date
    date_str format: YYYY-MM-DD
    """
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'sparkline': False,
        'date': date_str
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        return response.json()
    except Exception as e:
        print(f"Error for {date_str}: {e}")
        return None

def get_coin_market_cap(coin_id, from_date, to_date):
    """
    Get market cap data for a single coin with better error handling
    """
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    from_timestamp = int(datetime.strptime(from_date, '%Y-%m-%d').timestamp())
    to_timestamp = int(datetime.strptime(to_date, '%Y-%m-%d').timestamp())
    
    params = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get('market_caps', [])
    except Exception as e:
        print(f"Error getting market cap for {coin_id}: {e}")
        return []

def track_market_caps(historical_lists):
    """
    Enhanced market cap tracking with better error handling and progress tracking
    """
    today = datetime.now().strftime('%Y-%m-%d')
    all_market_caps = {}
    
    for date_label, assets in historical_lists.items():
        print(f"\nProcessing list: {date_label}")
        creation_date = date_label.split('_')[0]
        daily_totals = {}
        
        for i, (symbol, name, coin_id) in enumerate(assets, 1):
            if coin_id:
                print(f"Processing {symbol} ({i}/100)")
                market_caps = get_coin_market_cap(coin_id, creation_date, today)
                
                for timestamp_ms, market_cap in market_caps:
                    date = datetime.fromtimestamp(timestamp_ms/1000).strftime('%Y-%m-%d')
                    daily_totals[date] = daily_totals.get(date, 0) + market_cap
            
            time.sleep(1.2)  # Respect rate limits
        
        all_market_caps[date_label] = daily_totals
    
    return all_market_caps

def create_visualization(market_cap_df):
    """
    Create an interactive visualization using plotly
    """
    # Convert market cap to billions for better readability
    market_cap_df['total_market_cap_billions'] = market_cap_df['total_market_cap'] / 1e9
    
    # Create the figure
    fig = go.Figure()
    
    # Add a line for each historical list
    for date_label in market_cap_df['list_date_label'].unique():
        df_subset = market_cap_df[market_cap_df['list_date_label'] == date_label]
        fig.add_trace(go.Scatter(
            x=df_subset['tracking_date'],
            y=df_subset['total_market_cap_billions'],
            name=date_label,
            mode='lines',
            hovertemplate='Date: %{x}<br>Market Cap: $%{y:.2f}B<extra></extra>'
        ))
    
    # Update layout
    fig.update_layout(
        title='Historical Top 100 Lists - Market Cap Tracking',
        xaxis_title='Date',
        yaxis_title='Total Market Cap (Billions USD)',
        hovermode='x unified',
        legend_title='List Date',
        template='plotly_white',
        height=800
    )
    
    # Save the plot
    fig.write_html('market_cap_visualization.html')
    return fig

def main():
    # Define historical dates
    historical_dates = [
        "2018-01-01",
        "2020-01-01",
        "2022-01-01"
    ]
    
    # Get historical lists
    print("Creating historical lists...")
    historical_lists = {}
    for date in historical_dates:
        print(f"Fetching data for {date}")
        data = get_top_100_at_date(date)
        if data:
            assets = [(coin['symbol'].upper(), coin['name'], coin.get('id')) 
                     for coin in data if coin.get('id')]  # Only include coins with valid IDs
            date_label = f"{date}_t100"
            historical_lists[date_label] = assets
            time.sleep(1.2)
    
    # Track market caps with progress updates
    print("\nTracking market caps...")
    market_cap_data = track_market_caps(historical_lists)
    
    # Convert to DataFrame
    market_cap_records = []
    for date_label, daily_caps in market_cap_data.items():
        for date, market_cap in daily_caps.items():
            market_cap_records.append({
                'list_date_label': date_label,
                'tracking_date': date,
                'total_market_cap': market_cap
            })
    
    market_cap_df = pd.DataFrame(market_cap_records)
    
    # Save to CSV
    print("\nSaving results...")
    market_cap_df.to_csv('historical_lists_market_cap_tracking.csv', index=False)
    
    # Create visualization
    print("\nCreating visualization...")
    fig = create_visualization(market_cap_df)
    
    print("\nDone! Check 'historical_lists_market_cap_tracking.csv' and 'market_cap_visualization.html'")
    
    # Print some basic statistics
    print("\nBasic Statistics:")
    for date_label in market_cap_df['list_date_label'].unique():
        df_subset = market_cap_df[market_cap_df['list_date_label'] == date_label]
        print(f"\n{date_label}:")
        print(f"Average Market Cap: ${df_subset['total_market_cap'].mean()/1e9:.2f}B")
        print(f"Max Market Cap: ${df_subset['total_market_cap'].max()/1e9:.2f}B")
        print(f"Min Market Cap: ${df_subset['total_market_cap'].min()/1e9:.2f}B")

if __name__ == "__main__":
    main()

Creating historical lists...
Fetching data for 2018-01-01
Fetching data for 2020-01-01
Fetching data for 2022-01-01

Tracking market caps...

Processing list: 2018-01-01_t100
Processing BTC (1/100)
Error getting market cap for bitcoin: 401 Client Error: Unauthorized for url: https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1514777400&to=1731468600
Processing ETH (2/100)
Error getting market cap for ethereum: 401 Client Error: Unauthorized for url: https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from=1514777400&to=1731468600
Processing USDT (3/100)
Error getting market cap for tether: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/tether/market_chart/range?vs_currency=usd&from=1514777400&to=1731468600
Processing SOL (4/100)
Error getting market cap for solana: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/coins/solana/market_chart/range?vs_currenc

KeyboardInterrupt: 